# Generate sample NetCDF file following OG-1.0
J. Sevadjian, Nov 21, 2022
- Use Spray data to test making OG-1.0 NetCDF files
- Latest draft CDL: 
https://github.com/OceanGlidersCommunity/OG-format-user-manual/blob/main/sp041_20191205T1757-instrument-scalar_v2.cdl
- Latest draft manual: 
https://github.com/OceanGlidersCommunity/OG-format-user-manual/blob/main/OG_Format.adoc

## Imports


In [2]:
import numpy as np
import pandas as pd
import xarray as xr
import scipy 
import urllib3
import certifi
from erddapy import ERDDAP
import netCDF4
from netCDF4 import Dataset

## Load input data

In [16]:
# Use erddappy to fetch data for one spray glider trajectory

e = ERDDAP(server="https://gliders.ioos.us/erddap")
e.constraints = None
e.protocol = "tabledap"
e.dataset_id = "sp011-20221014T1612"

# Load glider data into xarray
ds = e.to_xarray(decode_times=False)
ds

<xarray.Dataset>
Dimensions:              (trajectory: 1, profile: 474, obs: 24917)
Coordinates:
  * trajectory           (trajectory) object 'sp011-20221014T1612'
    time                 (profile) float64 ...
    latitude             (profile) float64 ...
    longitude            (profile) float64 ...
    lat_uv               (profile) float64 ...
    lon_uv               (profile) float64 ...
    time_uv              (profile) float64 ...
    depth                (obs) float32 ...
Dimensions without coordinates: profile, obs
Data variables: (12/41)
    wmo_id               (trajectory) object ...
    profile_id           (profile) float64 ...
    u                    (profile) float64 ...
    v                    (profile) float64 ...
    trajectoryIndex      (profile) int32 ...
    rowSize              (profile) int32 ...
    ...                   ...
    temperature          (obs) float32 ...
    temperature_qc       (obs) float32 ...
    time_qc              (obs) float32 ...
    time_uv_qc           (obs) float32 ...
    u_qc                 (obs) float32 ...
    v_qc                 (obs) float32 ...
Attributes: (12/75)
    acknowledgement:                 Funded by National Oceanic and Atmospher...
    cdm_data_type:                   TrajectoryProfile
    cdm_profile_variables:           time_uv,lat_uv,lon_uv,u,v,profile_id,tim...
    cdm_trajectory_variables:        trajectory,wmo_id
    contributor_name:                Daniel Rudnick, Guilherme Castelao
    contributor_role:                Principal Investigator, Data Curator
    ...                              ...
    time_coverage_end:               2022-11-21T09:59:15Z
    time_coverage_start:             2022-10-14T16:48:45Z
    title:                           sp011-20221014T1612
    trajectory:                      sp011-20221014T1612
    Westernmost_Easting:             -126.1781
    xglider_type:                    profileObs

## Prep the data
- The spray data retrieved from ERDDAP is insufficient for the new OG-1.0 format
- Massage it to make it work for a sample file
- This section is specific to Spray data adjustments

In [5]:
modified_lons = []; modified_lats = []; modified_times = [];

for lon in ds["precise_lon"].data:
    if not np.isnan(lon):
        modified_lons.append(lon)
        last_real_value = lon
    else:
        modified_lons.append(last_real_value)

for lat in ds["precise_lat"].data:
    if not np.isnan(lat):
        modified_lats.append(lat)
        last_real_value = lat
    else:
        modified_lats.append(last_real_value)
        
# Precise time has some same values and Nans? Fixing
for idx, time in enumerate(ds["precise_time"].data):
    if not np.isnan(time):
        # Check for increasing...
        if idx>1 and time > modified_times[idx-1]:
            modified_times.append(time)
        else:
            # Resolve existing same time oddity
            modified_times.append(time+1)       
    else:
        # Insert fake time in place of nan
        modified_times.append(modified_times[idx-1] + 1)

test_list=modified_times
res = all(i < j for i, j in zip(test_list, test_list[1:]))  

# Finished with Spray input data tweaks 

## Write the NetCDF
Adjustments have been made to
1) Conform to CF conventions
2) Follow NCEI NetCDF templates
3) Display properly in Panoply (as GeoTraj)
4) Load into ERDDAP (as CF trajectory)

In [18]:
# Output Filename
path = 'nc_out_2022_11_21.nc'

# Write the file
with netCDF4.Dataset(path, mode='w', format='NETCDF4_CLASSIC') as ncout:

    # GLOBAL ATTRIBUTES
    
    # Start by populating with global attributes from erddap request
    for v in ds.attrs:
        if ds.attrs[v] is not None and v[:4] != "cdm_":
            setattr(ncout, v, ds.attrs[v])
    
    # Then reset these global attributes
    ncout.featureType = "trajectory";
    ncout.cdm_data_type = "Trajectory";
    
    # SETUP DIMENSIONS
    
    dims = {}
    
    # N_PARAM DIM 
    # Not clear what N_PARAM is meant to be. Leaving it out.
    # Is there are technical use case for the N_PARAM dimension? 
    # With it, I don't think the format is to CF spec and is perhaps overly complicated
    # I'm putting that same information in variable attributes for this example, 
    # This brings it to spec and simplifies it without losing the information.
    # Per NCEI templates: the instrument/sensor details should be specified as variables.
    # https://www.ncei.noaa.gov/data/oceans/ncei/formats/netcdf/v2.0/index.html
    
    # N_MEASUREMENTS DIM
    # Equiv. to 'obs' in CF docs.
    # Using the size of one data variable to specify the obs/measurements dimension
    dims['N_MEASUREMENTS'] = ncout.createDimension('N_MEASUREMENTS', ds.salinity.size)
    
    # TRAJECTORY DIM
    # CF docs require a trajectory dimension and variable. I am adding these.
    # There are options for specifying the trajectory dimension. 
    # The simplest is the number of trajectories in the file.
    dims['trajectory'] = ncout.createDimension('trajectory', 1)
    
    # CREATE VARIABLES
    
    variables = {}
    
    # Create Trajectory Variable 
    # Adding because required by CF
    # Does not read into ERDDAP without it because ERDDAP is expecting conformance to the CF trajectory format.
    variables['trajectory'] = ncout.createVariable(
        'trajectory', 
        'i4',
        'trajectory',)
    variables['trajectory'][:] = 1
    variables['trajectory'].cf_role = "trajectory_id"
    
    # CREATE COORDINATE VARIABLES (time, lat, lon, depth)
  
    # Time
    variables['TIME'] = ncout.createVariable(
        'TIME',
        ds['precise_time'].dtype,
        'N_MEASUREMENTS')
    variables['TIME'][:] = modified_times
    variables['TIME'].axis = "T"
    variables['TIME'].units = "seconds since 1970-01-01 00:00:00 UTC";
    variables['TIME'].calendar = "julian"
    variables['TIME'].standard_name = "time"
    variables['TIME']._fillValue = "-9999.0"
    variables['TIME'].long_name = "Time"
    
    # Latitude
    variables['LATITUDE'] = ncout.createVariable(
        'LATITUDE',
        ds['precise_lat'].dtype,
        'N_MEASUREMENTS')
    variables['LATITUDE'][:] = modified_lats
    variables['LATITUDE'].axis = "Y"
    variables['LATITUDE'].units = "degrees_north"
    variables['LATITUDE'].long_name = "Latitude"
    variables['LATITUDE'].standard_name = "latitude"
    
    # Longitude
    variables['LONGITUDE'] = ncout.createVariable(
        'LONGITUDE',
        ds['precise_lon'].dtype,
        'N_MEASUREMENTS')
    variables['LONGITUDE'][:] = modified_lons
    variables['LONGITUDE'].axis = "X"
    variables['LONGITUDE'].units = "degrees_east"
    variables['LONGITUDE'].long_name = "Longitude"
    variables['LONGITUDE'].standard_name = "longitude"
    
    # Depth
    variables['DEPTH'] = ncout.createVariable(
        'DEPTH',
        ds['depth'].dtype,
        'N_MEASUREMENTS')
    variables['DEPTH'][:] = ds['depth'].data
    variables['DEPTH'].axis = "Z"
    variables['DEPTH'].long_name = "Depth"
    variables['DEPTH'].standard_name = "Depth"
    variables['DEPTH'].units = "m"
    variables['DEPTH'].positive = "Down"
    
    # CREATE DATA VARS
    # Selecting these variables for this quick example
    # chlorophyll_a, density, dissolved_oxygen, dissolved_oxygen_qc
    
    # Chlorophyll Variable
    v = ds["chlorophyll_a"]
    variables[v.name] = ncout.createVariable('CHLA', v.dtype, 'N_MEASUREMENTS', fill_value=np.nan)
    variables[v.name][:]= v.data
    variables[v.name].coordinates = "TIME LATITUDE LONGITUDE DEPTH"
    variables[v.name].standard_name = v.standard_name
    variables[v.name].long_name = v.long_name
    variables[v.name].units = v.units
    variables[v.name].vocabulary = "https://vocab.nerc.ac.uk/collection/OG1/current/";
    variables[v.name].ancillary_variables = "CHLA_INSTRUMENT"
    
    # Density Variable
    v = ds["density"]
    variables[v.name] = ncout.createVariable('DENSITY', v.dtype, 'N_MEASUREMENTS', fill_value=np.nan)
    variables[v.name][:]= v.data
    variables[v.name].coordinates = "TIME LATITUDE LONGITUDE DEPTH"
    variables[v.name].standard_name = v.standard_name
    variables[v.name].long_name = v.long_name
    variables[v.name].units = v.units
    variables[v.name].vocabulary = "https://vocab.nerc.ac.uk/collection/OG1/current/";
    
    # Dissolved Oxygen Variable
    v = ds["dissolved_oxygen"]
    variables[v.name] = ncout.createVariable('DOXY', v.dtype, 'N_MEASUREMENTS', fill_value=np.nan)
    variables[v.name][:]= v.data
    variables[v.name].coordinates = "TIME LATITUDE LONGITUDE DEPTH"
    variables[v.name].standard_name = v.standard_name
    variables[v.name].long_name = v.long_name
    variables[v.name].units = v.units
    variables[v.name].vocabulary = "https://vocab.nerc.ac.uk/collection/OG1/current/";
    variables[v.name].ancillary_variables = "DOXY_QC DOXY_INSTRUMENT"
    
    # Dissolved Oxygen QC Variable
    v = ds["dissolved_oxygen_qc"]
    variables[v.name] = ncout.createVariable('DOXY_QC', v.dtype, 'N_MEASUREMENTS')
    variables[v.name][:]= v.data
    variables[v.name].coordinates = "TIME LATITUDE LONGITUDE DEPTH"
    variables[v.name].long_name = v.long_name
    
     
    # For sensor information you could use variables OR use attributes in geophysical vars
    # How to choose?
    # If there will be cases where a variable in a trajectory file will have more then one instrument OR
    # if these files will be aggregated and there is a potential for information loss upon aggregation
    # Otherwise attributes in the geopysical variables are sufficient and simplifies file structure
    
    # Demo Create Sensor/Instrument Variables
    # Containing the desired info that would have been in a param dim
    # https://www.ncei.noaa.gov/data/oceans/ncei/formats/netcdf/v2.0/index.html
    
    variables[v.name] = ncout.createVariable('CHLA_INSTRUMENT', 'i4', 'trajectory')
    variables[v.name][:]= 1
    variables[v.name].vocabulary = "https://docs.google.com/document/d/1dN90xkw9oCbLs0sPPhOmszdOjLpwcqxiK5mjeZP7abA/edit";
    variables[v.name].make_model = "ECO_FL"
    # Optional Attributes:
    # serial_number, calibration_date, factory_calibrated, user_calibrated, calibration_report, accuracy, valid_range, and precision
    
    variables[v.name] = ncout.createVariable('DOXY_INSTRUMENT', 'i4', 'trajectory')
    variables[v.name][:]= 1
    variables[v.name].vocabulary = "https://docs.google.com/document/d/1dN90xkw9oCbLs0sPPhOmszdOjLpwcqxiK5mjeZP7abA/edit";
    variables[v.name].make_model = "SEABIRD_SBE43F_IDO"
    # Optional Attributes:
    # serial_number, calibration_date, factory_calibrated, user_calibrated, calibration_report, accuracy, valid_range, and precision
